# SQL Azure Database Basic Monitor - Serverless Aggregation Queries

Basic monitoring queries to get information about your SQL Azure Database performance.

## Prerequisites

1. An Azure Synapse Workspace with a Synapse SQL Serverless Pool
2. Permission provided to the Serverless Pool Managed Identity to access the storage account
3. Data feeding into the storage account

## Initial Setup
These scripts are run to set up your initial views/access to the CSV files. These assume you have the proper permissions set up from your serverless pool to access your blob storage account.

### Credentials
These are the credentials and data source to access the CSV files, make sure to change the location to your storage account and container.
Comment these lines out if you have already created your sql credentials to access the data.

In [ ]:
CREATE MASTER KEY;

CREATE DATABASE SCOPED CREDENTIAL SynapseId
WITH IDENTITY = 'Managed Identity';

CREATE EXTERNAL DATA SOURCE SqlLogging
WITH (    LOCATION   = 'https://[youraccount].blob.core.windows.net/[yourcontainer]/',
          CREDENTIAL = SynapseId
);

### Performance View
This view will give you access to the Sql Performance Data retained from `sys.dm_db_resource_stats`

Make sure to edit the FIELDTERMINATOR to what you used as your LOG_FILE_CSV_SEPARATOR in your function settings.

In [29]:
CREATE OR ALTER VIEW SqlAzurePerformance
AS
SELECT CAST(end_time as DATETIME) as end_time, [avg_cpu_percent], [avg_data_io_percent], [avg_log_write_percent], [avg_memory_usage_percent], [xtp_storage_percent], [max_worker_percent], [max_session_percent], [dtu_limit], [avg_login_rate_percent], [avg_instance_cpu_percent], [avg_instance_memory_percent], [cpu_limit], [replica_role], CAST(perf.filepath(1) as DATE) as [file_date] FROM
    OPENROWSET(
        BULK 'performance-*.csv',
        FORMAT = 'CSV', 
        PARSER_VERSION = '2.0',
        FIELDTERMINATOR ='|',
        DATA_SOURCE = 'SqlLogging',
        ROWSET_OPTIONS = '{"READ_OPTIONS":["ALLOW_INCONSISTENT_READS"]}'
    )
    WITH (
        [end_time] VARCHAR(25) COLLATE Latin1_General_100_BIN2_UTF8,
        [avg_cpu_percent] DECIMAL(8,2),
        [avg_data_io_percent] DECIMAL(8,2),
        [avg_log_write_percent] DECIMAL(8,2),
        [avg_memory_usage_percent] DECIMAL(8,2),
        [xtp_storage_percent] DECIMAL(8,2),
        [max_worker_percent] DECIMAL(8,2),
        [max_session_percent] DECIMAL(8,2),
        [dtu_limit] INT,
        [avg_login_rate_percent] DECIMAL(8,2),
        [avg_instance_cpu_percent] DECIMAL(8,2),
        [avg_instance_memory_percent] DECIMAL(8,2),
        [cpu_limit] INT,
        [replica_role] INT
    ) 
    as [perf]

Commands completed successfully.

Total execution time: 00:00:08.019

In [18]:
CREATE OR ALTER VIEW SqlAzureExtendedPerformance
AS
SELECT 
        CAST([reading_time] as DATETIME) as [reading_time],
        [session_id],
        [request_id],
        [ecid],
        [blocking_session_id],
        [blocking_ecid],
        [task_state],
        [wait_type],
        [wait_duration_ms],
        [wait_resource],
        [resource_description],
        [last_wait_type],
        [open_trans],
        [transaction_isolation_level],
        [is_user_process],
        [request_cpu_time],
        [request_logical_reads],
        [request_reads],
        [request_writes],
        [memory_usage],
        [session_cpu_time],
        [session_reads],
        [session_writes],
        [session_logical_reads],
        [total_scheduled_time],
        [total_elapsed_time],
        CAST([last_request_start_time] as DATETIME) as [last_request_start_time],
        CAST([last_request_end_time] as DATETIME) as [last_request_end_time],
        [session_row_count],
        [prev_error],
        [open_resultsets],
        [request_total_elapsed_time],
        [percent_complete],
        [est_completion_time],
        [tran_name],
        CAST([transaction_begin_time] as DATETIME) as [transaction_begin_time],
        [tran_type],
        [tran_state],
        CAST([request_start_time] as DATETIME) as [request_start_time],
        [request_status],
        [command],
        [plan_handle],
        [sql_handle],
        [statement_start_offset],
        [statement_end_offset],
        [database_id],
        [user_id],
        [executing_managed_code],
        [pending_io_count],
        CAST([login_time] as DATETIME) as [login_time],
        [host_name],
        [program_name],
        [host_process_id],
        [client_version],
        [client_interface_name],
        [login_name],
        [nt_domain],
        [nt_user_name],
        [net_packet_size],
        [client_net_address],
        [most_recent_sql_handle],
        [session_status],
        [scheduler_id],
        [group_id],
        [context_info],
        CAST(perf.filepath(1) as DATE) as [file_date] FROM
    OPENROWSET(
        BULK 'extended-performance-*.csv',
        FORMAT = 'CSV', 
        PARSER_VERSION = '2.0',
        FIELDTERMINATOR ='|',
        DATA_SOURCE = 'SqlLogging',
        ROWSET_OPTIONS = '{"READ_OPTIONS":["ALLOW_INCONSISTENT_READS"]}'
    )
    WITH (
        [reading_time] nvarchar(25) COLLATE Latin1_General_100_BIN2_UTF8,
        [session_id] [smallint],
        [request_id] [int],
        [ecid] [int],
        [blocking_session_id] [smallint],
        [blocking_ecid] [int] ,
        [task_state] nvarchar(15) COLLATE Latin1_General_100_BIN2_UTF8,
        [wait_type] [nvarchar](60) COLLATE Latin1_General_100_BIN2_UTF8,
        [wait_duration_ms] [bigint],
        [wait_resource] [nvarchar](40) COLLATE Latin1_General_100_BIN2_UTF8,
        [resource_description] [nvarchar](140) COLLATE Latin1_General_100_BIN2_UTF8,
        [last_wait_type] [nvarchar](50) COLLATE Latin1_General_100_BIN2_UTF8,
        [open_trans] [int],
        [transaction_isolation_level] [varchar](30) COLLATE Latin1_General_100_BIN2_UTF8,
        [is_user_process] [bit],
        [request_cpu_time] [int],
        [request_logical_reads] [bigint],
        [request_reads] [bigint],
        [request_writes] [bigint],
        [memory_usage] [int],
        [session_cpu_time] [int],
        [session_reads] [bigint],
        [session_writes] [bigint],
        [session_logical_reads] [bigint],
        [total_scheduled_time] [int],
        [total_elapsed_time] [int],
        [last_request_start_time] nvarchar(25) COLLATE Latin1_General_100_BIN2_UTF8,
        [last_request_end_time] nvarchar(25) COLLATE Latin1_General_100_BIN2_UTF8,
        [session_row_count] [bigint],
        [prev_error] [int],
        [open_resultsets] [int],
        [request_total_elapsed_time] [int],
        [percent_complete] [decimal](5, 2),
        [est_completion_time] [bigint],
        [tran_name] [nvarchar](24),
        [transaction_begin_time] nvarchar(25) COLLATE Latin1_General_100_BIN2_UTF8,
        [tran_type] [varchar](15) COLLATE Latin1_General_100_BIN2_UTF8,
        [tran_state] [varchar](15) COLLATE Latin1_General_100_BIN2_UTF8,
        [request_start_time] nvarchar(25) COLLATE Latin1_General_100_BIN2_UTF8,
        [request_status] [nvarchar](15) COLLATE Latin1_General_100_BIN2_UTF8,
        [command] [nvarchar](16) COLLATE Latin1_General_100_BIN2_UTF8,
        [plan_handle] [varbinary](64),
        [sql_handle] [varbinary](64),
        [statement_start_offset] [int],
        [statement_end_offset] [int],
        [database_id] [smallint],
        [user_id] [int],
        [executing_managed_code] [bit],
        [pending_io_count] [int],
        [login_time] nvarchar(25) COLLATE Latin1_General_100_BIN2_UTF8,
        [host_name] [nvarchar](20) COLLATE Latin1_General_100_BIN2_UTF8,
        [program_name] [nvarchar](50) COLLATE Latin1_General_100_BIN2_UTF8,
        [host_process_id] [int],
        [client_version] [int],
        [client_interface_name] [nvarchar](30) COLLATE Latin1_General_100_BIN2_UTF8,
        [login_name] [nvarchar](30) COLLATE Latin1_General_100_BIN2_UTF8,
        [nt_domain] [nvarchar](30) COLLATE Latin1_General_100_BIN2_UTF8,
        [nt_user_name] [nvarchar](20) COLLATE Latin1_General_100_BIN2_UTF8,
        [net_packet_size] [int],
        [client_net_address] [varchar](20) COLLATE Latin1_General_100_BIN2_UTF8,
        [most_recent_sql_handle] [varbinary](64),
        [session_status] [nvarchar](15) COLLATE Latin1_General_100_BIN2_UTF8,
        [scheduler_id] [int],
        [group_id] [int],
        [context_info] varbinary(128)
    ) 
    as [perf]

Commands completed successfully.

Total execution time: 00:00:07.623

## Example Queries
Here are some example queries to start using the performance data. 

When writing your own queries, make sure to timebox your data using `file_date` as the predicate as this will use partition elimination to improve your query performance.

### Top 5 instances of CPU Usage over the last week

In [8]:
SELECT TOP 5 end_time, avg_cpu_percent, avg_data_io_percent, avg_log_write_percent
FROM SqlAzurePerformance WHERE file_date > DATEADD(day, -7, CURRENT_TIMESTAMP) ORDER BY avg_cpu_percent DESC

Statement ID: {464B791B-F8E5-461D-97CA-621BAB28205E} | Query hash: 0xA535616A1B83920E | Distributed request ID: {309857E9-B594-4F96-ADCB-C71689548CDE}. Total size of data scanned is 3 megabytes, total size of data moved is 1 megabytes, total size of data written is 0 megabytes.

(5 rows affected)

Total execution time: 00:00:29.896

end_time,avg_cpu_percent,avg_data_io_percent,avg_log_write_percent
2022-11-07 22:21:00.000,67.00,0.00,NULL
2022-11-07 22:22:00.000,67.00,0.00,NULL
2022-11-07 22:23:00.000,67.00,0.00,NULL
2022-11-07 22:24:00.000,67.00,0.00,NULL
2022-11-07 22:20:02.000,67.00,0.00,NULL


In [19]:
SELECT * FROM SqlAzureExtendedPerformance

Statement ID: {6B37D7A3-6026-427C-91B6-2A2E1025E1F6} | Query hash: 0x796F3177A2F3FE10 | Distributed request ID: {0F78D9F4-89A0-40ED-A268-A45BA14A2A3D}. Total size of data scanned is 1 megabytes, total size of data moved is 0 megabytes, total size of data written is 0 megabytes.

Total execution time: 00:00:11.294

reading_time,session_id,request_id,ecid,blocking_session_id,blocking_ecid,task_state,wait_type,wait_duration_ms,wait_resource,resource_description,last_wait_type,open_trans,transaction_isolation_level,is_user_process,request_cpu_time,request_logical_reads,request_reads,request_writes,memory_usage,session_cpu_time,session_reads,session_writes,session_logical_reads,total_scheduled_time,total_elapsed_time,last_request_start_time,last_request_end_time,session_row_count,prev_error,open_resultsets,request_total_elapsed_time,percent_complete,est_completion_time,tran_name,transaction_begin_time,tran_type,tran_state,request_start_time,request_status,command,plan_handle,sql_handle,statement_start_offset,statement_end_offset,database_id,user_id,executing_managed_code,pending_io_count,login_time,host_name,program_name,host_process_id,client_version,client_interface_name,login_name,nt_domain,nt_user_name,net_packet_size,client_net_address,most_recent_sql_handle,session_status,scheduler_id,group_id,context_info,file_date


: Msg 13812, Level 16, State 1, Line 1
Bulk load data conversion error (type mismatch or invalid character for the specified codepage) for row starting at byte offset 0, column 42 (plan_handle) in data file extended-performance-2022-11-07.csv.